In [6]:
'''
Reference:
Example script from Keras Github Repository keras/examples/lstm_text_generation.py
https://github.com/keras-team/keras

Same configuration as the vanilla LSTM char-RNN for Donna Summer.
Trained on song lyrics of Gordon Lightfoot only
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
#text = io.open(path, encoding='utf-8').read().lower()
import pandas as pd
lyrics_data = pd.read_csv('songdata.csv')

# Gordon Lightfoot
lyrics_data_gordon = lyrics_data[lyrics_data.artist == 'Gordon Lightfoot']['text']

lyrics_concat_gordon = u''
for i in lyrics_data_gordon.index:
    lyrics_concat_gordon += lyrics_data_gordon[i]
    
print("input chars length")
print(len(lyrics_concat_gordon))
text = lyrics_concat_gordon
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

input chars length
230377
corpus length: 230377
total chars: 73
nb sequences: 76779
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
76779/76779 [==============================] - 39s - loss: 2.0612    

----- diversity: 0.2
----- Generating with seed: " if you don't say may I,  
Then I'll mar"
 if you don't say may I,  
Then I'll mare  
And the way  
In the stime  
  
In the stide there  
  
In the way  
In the way  
  
I could day  
When the stome  
In the stome  
  
In the stome  
  
I way  
In the stide  
  
Ind the stide  
  
In the way  
  
I was and the way  
In the way  
In the stain  
In the still there  
In the way  
  
In the way  
In the way  
  
In the way  
In the stime  
In the way  
And the sead  
In the still 

----- diversity: 0.5
----- Generating with seed: " if you don't say may I,  
Then I'll mar"
 if you don't say may I,  
Then I'll mare  
In the stand  
When the sead and it the ssee  
In the way  
In the ru

ime I've wandered I always have returned I just  
When I'm nots out mind?  
She you have surble tray bow you nebisest upon  
Eventy five glo sang to torolnars  
Heasts for murnin' and a deak wond I've feel hat his again  
Little Whome any, I'd cikien id not for the swown  
I take themsers a matth to life  
Then These Fascon of times upon coucsed black ifs my lut  
With the cown , find me rain  
Then goen', wassed to cam

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log


an lothin' is awa

----- diversity: 1.2
----- Generating with seed: "ime I've wandered I always have returned"
ime I've wandered I always have returned on litels faotir thisk the unts, dlyand  
Brokmershiess con my moss for try to a foichif'e on therot bown bus in  
Tut down With anything she dime  
Willy not be to time  
Who id like you'-rokes gon, to feels stow fromb bury  
  
It list stars authe me 'til for my brink  
There's mon't gir hay by somew'll gros foodion shep on, talkin'  
For I mauch your eyes me's piess  
We'll all tan, colls  
Al

--------------------------------------------------
Iteration 5
Epoch 1/1
76779/76779 [==============================] - 37s - loss: 1.3407    

----- diversity: 0.2
----- Generating with seed: "s will all come true  
In the sunset the"
s will all come true  
In the sunset the street  
It's there we got a say  
And the sun in the sun  
I can wild the wild  
And the strandy face to go  
It's all the streets and the way and the sun  
There's a fe

And sever I can shine to the new to velive coar dream  
Bohed help yfesto wake Starsin'  
It will veished in the bright my pace the sweet to you  

I

--------------------------------------------------
Iteration 12
Epoch 1/1
76779/76779 [==============================] - 37s - loss: 1.1349    

----- diversity: 0.2
----- Generating with seed: "ver fall down  
If you pity the stranger"
ver fall down  
If you pity the stranger  
I was all alone in the world  
  
When I was the street  
And the stand of the strentlors  
And the stand his way and stars  
I was a man what day  
And the water, the world and sear  
There's a might to get me  
I was all alone in the world  
  
There's a step back  
  
If you can see it the sky  
The sky is for and step back  
  
I was all alone in the water with a might  
  
She lived to the

----- diversity: 0.5
----- Generating with seed: "ver fall down  
If you pity the stranger"
ver fall down  
If you pity the stranger fall

I can see, me down in my love y

...

Who like the way you wasted a shout  
I do not say to can she too  
I want to change  
I was where the way to be a fever  
Long a fine soon my way you want foul no may the rester  
This and brottle wall  
I think I  
So dreams everyone is

----- diversity: 1.0
----- Generating with seed: "ing you can't waste  
Then listen bud, l"
ing you can't waste  
Then listen bud, like through he haps that dust from the land  
And so easy side of the rain,  
Thank is in my hand work see  
And prettle a reach eyd of your scole  
To the way now is hanger  
feet you winin' me in, he waves around or nive there home  
  
Mama song the window  
Some ho words sofelyour  
And if the eys in my high in the slee  
And when better on my own hare  
I unreas where ya sho's in my free  
To

----- diversity: 1.2
----- Generating with seed: "ing you can't waste  
Then listen bud, l"
ing you can't waste  
Then listen bud, long hearthide hur with cander time could befort home  
  
Where it is trouble do by anything  

76779/76779 [==============================] - 44s - loss: 0.9355    

----- diversity: 0.2
----- Generating with seed: "nd on 'til all sadness is gone  
And if "
nd on 'til all sadness is gone  
And if I can't folls that you get thin dawn  
  
I want to know that we knew  
I would see you a feel the water, who diprens  
And the same of the stand is tor when the rain  
I would cry to the road through the rest  
I'm stidn snever ground a mall town  
I would see you can't be all I done  
And the same of the street  
  
I want to go a place to be  
Anyow the same of the street  
  
I want to go .  


----- diversity: 0.5
----- Generating with seed: "nd on 'til all sadness is gone  
And if "
nd on 'til all sadness is gone  
And if I can't seemon more to the roon  
  
If I am englass and still make it alright  
I got a fen the light of a place this don't say  
And your home and the true warm and the country sharoon time  
And the roseny way a way the she hand and wish bother  
I belong some

And the boy in the streets at the street  
A long and steal that you been  
  
I hope you can't call  
I think you go and I can't sleep  
The times and I can't sleep in the dark  
And the child that it is my heart  
And the blue slip and 

----- diversity: 0.5
----- Generating with seed: "ld me, the town where I was born, she al"
ld me, the town where I was born, she all all must be wonder way to be  
Any way I could fried the summer with a time  
And the people way to chang mey  
His conner that people was to were the bad big cuss  
  
And it somewhere and made  
  
That we ain so but the broken slip con't stay  
That we get it  
It more to be on the dark  
And the child town less lies The played  
Deedlee-dumb", deedlee-dumbppliggts  
And the morning on the st

----- diversity: 1.0
----- Generating with seed: "ld me, the town where I was born, she al"
ld me, the town where I was born, she all never be and Is it regrums, only wand in the fave that before  
Make my way for the vain  
R

Is there a better man thouk ptier this proves seal  
We grow of a long who knew for hin to looksice  
It's ood maskee me do you  
Behood you gove houls raug  
She teld this hie my blue she tall again  
 fire in the mone  
There's on let through a  
And is ettooss that lay, you'll be no zyied tipen in my eye  
us ils's my fichandus love for my kilszed  
alls mady ucunina the finne blaget hand  
I was the tree down, I t

--------------------------------------------------
Iteration 48
Epoch 1/1
76779/76779 [==============================] - 40s - loss: 0.8836    

----- diversity: 0.2
----- Generating with seed: " follow or to find her  
  
She was some"
 follow or to find her  
  
She was some say that's this  
It's all all my brake her the way  
If you can always

The fire and in you was a friend  
It am the way the dollar  
In the warm from the faves, gonna grow of story  
They said fithing a thing the sea  
When the child the blues and the faves bloud  
Where the woman dide  
If you c

Tlay I walk

--------------------------------------------------
Iteration 55
Epoch 1/1
76779/76779 [==============================] - 38s - loss: 0.8619    

----- diversity: 0.2
----- Generating with seed: " so free  
There dwells a maid with a he"
 so free  
There dwells a maid with a heave will stayed  
And the windors of the start  
And the way you waste  
I was on the streets  
I was we air we air we a place  
The way is my eye  
  
I would long to be on you  
There was think of me  
And the wildoride with a true  
And the sounds the streets at the song  
And the love thought in the world  
And the stat of my love that somewhere  
  
I was born all the way you way  
  
If you 

----- diversity: 0.5
----- Generating with seed: " so free  
There dwells a maid with a he"
 so free  
There dwells a maid with a heart canders  
I like a place her headles  
I think that we knew ou heade, I'm a-gjwat  
And the wisded of winne have I was came to be  
And the sun goestway  
  
If you saw no 

In [7]:
# sentence 1

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Loneliness is always looking for a frien'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Loneliness is always looking for a frien"
Loneliness is always looking for a friend  
  
If you love me any

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log


one who laught  
The way you wrong that you belong side  
The water, he knew he played and never saw  
It was the dawn to the band down  
If you need is to be  
As I was be was me  
She don't know when the sea  
I'm only love way barder was too all her this  
There's a long what we air you see  
And the soundsion old man life the sea  
If you need the waters to her  
I was all my drink  
A vome love is not towns and need is  
The teld wind on the bree don't be the same  
And the soundsion of the life that I do  
I don't know when the bree  
In the words to be relied  
I'll dang the sound, some life that you say  
If you can seems on the shoulder  
The evenin' is my day  
If you can seems on the shoulder  
The soundsion old last turn to the way you wasjes

Is there a better may lot big closs  
And the sound, seet you when I can't do ever tonight  
What it's my fin

----- diversity: 0.5
----- Generating with seed: "Loneliness is always looking for a frien"
Loneliness is always looking fo

In [8]:
# sentence 2

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "We go hard or we go home \nWe gon' do thi"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "We go hard or we go home 
We gon' do thi"
We go hard or we go home 
We gon' do this wire  
The waters in the water, he knew  
A let it ride with a mill it all around  
  
I'm stime the same to see  
And the way you can see  
  
I'm on the sand of this hang to her  
I'd stidring, had it so face  
I can't sleep and by  
The way you cries and the waters fall  
I was born and the band where the words  
And the sun greats full out on a promise  


/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log


The way you can see it in my morning  
I was on the sard of the little way  
As I don't know when the band  
And the latth the dusty said  
  
The waters in the evenin' is  
  
I'd stall I'd be  
And the way you wasted as the first down  
The teld it so face  
It's a long way back and need is  
The way you cries and never saw  
It was the dawn the way you cries and the waters begin to be  
And the lovers and the blade down the teld  
I do no more that's the story  
The way you need is  
It's time to realing throus sing  
The way yo

----- diversity: 0.5
----- Generating with seed: "We go hard or we go home 
We gon' do thi"
We go hard or we go home 
We gon' do this wirse  
The air was my face the sea  
If you're at reach to get mey  
I've got that I sur surmoring to side  
We been too long together, my trustle street  
A let it make the chaine  
  
In the world tonight  
I've got the lovers and starst  
Coblood of a sun more of a smid blue  
The heavedling, like the morning stand  
And 

In [9]:
# sentence 3

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "At first I was afriad, I was petrified \n"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "At first I was afriad, I was petrified 
"
At first I was afriad, I was petrified 

If the road t

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log


he sound, seet you when I can't do and the band  
All the same the worrswills  
But not so face the river  
I would cryin' has but by the blue  
If you can seem to be the song  
The seep it sure  
The way you can see  
  
I'm on the tide wand a reach out  
The waters fly me a way this sight  
I stood that you been  
I can seems on the song  
I'm on his more to start seeme  
And the lovers and the blade away  
And the sound, as steal in the evenin' is  
I can see it in my mother stope like this  
So day that's when the season  
A like the seas sight  
A little older of this sight  
I was born and the blade away  
But the summer side of the barn could  
The teld of my mind the world  
  
If you something should home  
  
It it a till I can't sleep and never spole  
I've got the dreams  
I've got the dreams  
I've got the like to try and by  
If you want to know what do you lo

----- diversity: 0.5
----- Generating with seed: "At first I was afriad, I was petrified 
"
At first I was afria

In [10]:
# sentence 4

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Young Money, Nicki Minaj, Justin  \n  \nSh"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Young Money, Nicki Minaj, Justin  
  
Sh"
Young Money, Nicki Minaj, Justin  
  
She walks sures the way you wasten down  
If you should not signs 

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log


 
The eng of a sime that I saw about streal  
A little older I didn't say I'd straid  
I like this home is not torse  
I've got there  
The waters in the water, I didn't want to do

If you love you feet a dream  
When you think about the way you wast to her  
I'm on my face the shines  
Becaify come son some life feelin' in the water, like the finar

There was the waters fater, we got to me  
I was do you want to be peace  
  
I was born and the life that I've me do and the bree  
In the wild a same  
The waters be a tricked walk burn and need me"  
I don't know I'll be on  
I was all my friend in the evenin' is  
I can see it in your eye  
  
I don't know that you say  
If you and cander when the child shake  
The water way back and need it friend  
  
If you need is times  
And the seas a shore in the waters  
If you can s

----- diversity: 0.5
----- Generating with seed: "Young Money, Nicki Minaj, Justin  
  
Sh"
Young Money, Nicki Minaj, Justin  
  
She said, "Deedlee-dumb, dee-dee

In [11]:
# sentence 5

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Life is to be lived on the positive tip "
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Life is to be lived on the positive tip "
Life is to be lived on the positive tip  
I'd stand it a beaut  
I can't sleep and by the way you wasjest want to the summer way  
The seep to my right rest  
The waters fall at the rain  
I was all my drinWullfuiWe have all my by  
When I can't find the song  
And the sun grows a sand about sing  
The sky above you seen th

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log


e way  
If you love you seen the seas  
And the seas the blade away  
But the summer side of the barning  
And the sun grows a little love  
The sounds of the life tines  
And the sounds of a song  
The move the bandoo her land of the love I'd come to sea  
And the waters in the water, he played a mape  
The water, had much amank and I'm care  
  
If you something the river wond best  
The teld of my love was all around  
  
If you need is fire  
If I could thilling it away  
I'd been down you seen done  
And the same a bot of the song  
I'm some was the dawn fllace this hang sea  
A like you will really lov

----- diversity: 0.5
----- Generating with seed: "Life is to be lived on the positive tip "
Life is to be lived on the positive tip  
It it shines and it's blue  
  
The way and I'll care on you have you seen  
I was be stain mind  
I'll be all with a million trains  
We cannever can't plain  
It's time and a way I didn't want to pass  
Bent is the road of gray and the waters fall

In [12]:
# sentence 6

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Another night I'm waiting for you  \nWait"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Another night I'm waiting for you  
Wait"
Another night I'm waiting for you  
Waiting in the dawn is now  
I don't matter if I could do  

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log



If you need is lake a morning side  
The blade of a sun anough is say  
It's all time in the dawn  
And the bandles of the line  
And the waters in the dawn the water, I didnong to sea  
And the blue seat who laught we get  
We more the water way a day the sea  
If you not so face  
I'll never seail side the way you wasjes

Is there a better may the bleed is the shore  
Till I know will all make the sunse  
And the lovers and the blade away  
And the waters of the summer was too looks  
Will you pless you  
I'm leavin' here it in my mind  
I don't know when the bree don't day  
If you want to go, were the dawn is noide  
The waters for a sun achose is to be  
As I was all my drinWullfees  
And the waters for the shore  
  
If you something that I could do  
It's time and it start be home  
And the waters of the line  
And the sounds

----- diversity: 0.5
----- Generating with seed: "Another night I'm waiting for you  
Wait"
Another night I'm waiting for you  
Waiting but well the worl

In [13]:
# sentence 7

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Back track, listen, stop that  \nBefore e'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Back track, listen, stop that  
Before e"
Back track, listen, stop that  
Before endlesomes of a sun  


/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log


There's a man  
  
This it's all tho weround of a sun  
There's a slees in a place the world  
  
I don't know it shines  
And the waters in the dawn the water, dises on the sgair  
In the name and the blade away  
And the lovers and the blade away  
But not broken in my window  
It's time to be on the soul   
Well I fust for a summertime done man side  
The song the seas side of the barn could  
I knewt lies and need is  
The way you write as the sand of time  
To be oncfoue and in sit  
I don't know I can see  
  
The song the life that I sur  
The evenin' is my said she made a man din' to be  
And the summer with a way this sight  
I stormer to me a dain  
  
The way you cries and the waters beginne  
The per was make it a place the waters bedows  
And I need is  
The teld wind on the shoulder  
  
The way you long to hear the water, did be  
The waters fater, we 

----- diversity: 0.5
----- Generating with seed: "Back track, listen, stop that  
Before e"
Back track, listen, stop th

In [14]:
# sentence 8

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'How can a non-musician discuss the futur'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "How can a non-musician discuss the futur"
How can a non-musician discuss the future all time the earth  
The waters fly the way you wast you river  
I was on the shoulder

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log


  
The soundsong that we knew, she don't know  
A little oldo make it a side with a millight  
The evenin' is a step and the waters fall  
  
I don't matter it so face  
I can seems on the sand of times  
Take the blue seat whore the worrsching out old by  
If you can seem and the waters beginne  
The waters fall to me  
I was born and the blade away  
And the lake and the waters in the dawn  
If you not so my swim back and the way  
If you want to know that you'll make you see  
It's a long way back and need  
  
I don't know it away in the evenin' is  
I can see, the waters best  
The evenin' is my said she made a man be  
The soundsion or this hand spill  
I was all my dreams  
If you can seem and the blade her all the way  
And if I do do it a say  
In the waters in the dawn the water, I can't sle

----- diversity: 0.5
----- Generating with seed: "How can a non-musician discuss the futur"
How can a non-musician discuss the future all all my dyidd  
With a kissin' to the river  
The

In [15]:
# sentence 9

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Hi out there. This is Shaun Goldberg and'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Hi out there. This is Shaun Goldberg and"
Hi out there. This is Shaun Goldberg and I straight to the waters  
And the water with a way this sook of my mind  
I wa

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log


s been and the song the child  
We been too long song  
The sounds of the river wind  
And the soundsion of my window  
If you need is to be there  
I'll died it a sang things that I sure  
The water, have you seen the way you wasjest want to the way  
If you want to know where the worrswand as the brest  
As I know I cauld a mild  
With her so strand  
  
The teld her recin  
The way you cries, what do you where I can't sleep  
I can't sleep and never seen the right wind  
And it sea my bed tonight  
Whate is for the summer way  
This is my her streets at the rain  
So day it's a red undon  
And the bandoo but the way you wasjest  
The teld wind on the song they can't we for the blue  
If you would be do you face  
I was on the sard of the looks and the waters to sid  
And the bandoo but the way you wasjes, 

----- diversity: 0.5
----- Generating with seed: "Hi out there. This is Shaun Goldberg and"
Hi out there. This is Shaun Goldberg and one Rosanna  
I like the way you can't danger

In [16]:
# sentence 10

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Whose eyes in my behind  \nI don't recogn"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Whose eyes in my behind  
I don't recogn"
Whose eyes in my behind  
I don't recognes and need it dreams  
I'm on h pas the s

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in log


trand  
  
The water way back again the way you wasjest  
The man was the rain  
Oh sulms on the shoul, it stope lies  
And the sounds of a song  
The was all my dreams will never sleep  
I can't step back a hand to the boat to the waters believer  
I was born and the band where the well  
And the sound of the river  
I was bough I'd do and in sill  
The teld her recins a child it along watch it alring  
A lot again she made in the shore  
Till I know I'd stripp in the shoulder  
The soundsion old lafty as the summer to the sea  
I'd stall I'd do you what you get to be  
And the seas a sun anourdarts what was this sight  
A long and you can't do and the blade where the words  
And the barn cool to be  
And the lake and it's the evenin' is my said  
  
The waters in the dawn is noise  
I was bod way I could do  
If you need is a summer that I coul

----- diversity: 0.5
----- Generating with seed: "Whose eyes in my behind  
I don't recogn"
Whose eyes in my behind  
I don't recogned and I